In [223]:
import polars as pl
import altair as alt       

research_ethics_system = pl.read_csv("data/4_KIOST_research_ethics_system.csv")

In [224]:
def count_y(name):
    return research_ethics_system.group_by(pl.col(name).alias("Y")).len().rename({"len":name})   

def count_list_y(columns, select):
    df = pl.DataFrame({"Y":select})
    for col in columns:
        count = research_ethics_system.group_by(pl.col(col).alias("Y")).len().rename({"len":col})
        df = df.join(count, on="Y", how="left")
    return df

def make_chart(columns):
    charts = []
    for col in columns:
        chart = alt.Chart(research_ethics_system).mark_bar().encode(
            x=f"{col}:O",
            y=f"count(ID):N",
            color=f"{col}:O"
        ).properties(
            title=f"Distribution of {col}"
        )
        charts.append(chart)
    return alt.hconcat(*charts).resolve_scale(
        y='shared'  # Synchronize the y-axis scale across all charts
    )

In [225]:
# 
# 가. KIOST의 연구윤리 관련 규정에 대한 인지 여부
# 
# Loop through each column and create a bar chart
columns = {
    "Q21_1":"연구윤리_지침", 
    "Q21_2":"동물실험_윤리",
    "Q21_3":"연구노트_관리", 
    "Q21_4":"연구과제_관리지침", 
    "Q21_5":"공직자_이해충돌_방지_제도_운영_지침"
}

system_recognition = count_list_y(columns.keys(), [1, 2]).rename(columns)
system_recognition.write_csv("figure/4/가_KIOST의_연구윤리_관련_규정에_대한_인지_여부.csv", include_bom=True)

print(system_recognition)

chart = make_chart(columns)
chart.show()
chart.save("figure/4/가_KIOST의_연구윤리_관련_규정에_대한_인지_여부.svg")

shape: (2, 6)
┌─────┬───────────────┬───────────────┬───────────────┬───────────────────┬────────────────────────┐
│ Y   ┆ 연구윤리_지침 ┆ 동물실험_윤리 ┆ 연구노트_관리 ┆ 연구과제_관리지침 ┆ 공직자_이해충돌_방지_  │
│ --- ┆ ---           ┆ ---           ┆ ---           ┆ ---               ┆ 제도_운영_지침         │
│ i64 ┆ u32           ┆ u32           ┆ u32           ┆ u32               ┆ ---                    │
│     ┆               ┆               ┆               ┆                   ┆ u32                    │
╞═════╪═══════════════╪═══════════════╪═══════════════╪═══════════════════╪════════════════════════╡
│ 1   ┆ 9             ┆ 38            ┆ 8             ┆ 17                ┆ 33                     │
│ 2   ┆ 158           ┆ 129           ┆ 159           ┆ 150               ┆ 134                    │
└─────┴───────────────┴───────────────┴───────────────┴───────────────────┴────────────────────────┘


alt.HConcatChart(...)

In [226]:
# 
# 나. KIOST의 연구윤리 규정의 내용 파악 정도
# 
columns = {
    "Q22_1":"연구윤리_지침", 
    "Q22_2":"동물실험_윤리",
    "Q22_3":"연구노트_관리", 
    "Q22_4":"연구과제_관리지침", 
    "Q22_5":"공직자_이해충돌_방지_제도_운영_지침"
}
counting = count_list_y(columns.keys(), [1,2,3,4,5]).drop("Y")

proportion = counting.select(
    (pl.col("Q22_1") / (pl.col("Q22_1").sum())),
    (pl.col("Q22_2") / (pl.col("Q22_2").sum())),
    (pl.col("Q22_3") / (pl.col("Q22_3").sum())),
    (pl.col("Q22_4") / (pl.col("Q22_4").sum())),
    (pl.col("Q22_5") / (pl.col("Q22_5").sum())),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

counting = counting.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

mean = research_ethics_system.select(
    pl.col("Q22_1").mean(),
    pl.col("Q22_2").mean(),
    pl.col("Q22_3").mean(),
    pl.col("Q22_4").mean(),
    pl.col("Q22_5").mean(),
).rename(columns).transpose(include_header=True).rename({"column_0":"mean"})

std = research_ethics_system.select(
    pl.col("Q22_1").std(),
    pl.col("Q22_2").std(),
    pl.col("Q22_3").std(),
    pl.col("Q22_4").std(),
    pl.col("Q22_5").std(),
).rename(columns).transpose(include_header=True).rename({"column_0":"std"})

all = counting.join(
    proportion, on="column", how="inner"
).join(
    mean, on="column", how="inner"
).join(
    std, on="column", how="inner"
)

print(all)

all.write_csv("figure/4/나_KIOST의_연구윤리_규정의_내용_파악_정도.csv", include_bom=True)

chart = make_chart(columns)
chart.show()
chart.save("figure/4/나_KIOST의_연구윤리_규정의_내용_파악_정도.png")

shape: (5, 13)
┌──────────────┬───────┬──────────┬──────────┬───┬─────────────┬─────────────┬──────────┬──────────┐
│ column       ┆ count ┆ column_1 ┆ column_2 ┆ … ┆ column_3_ri ┆ column_4_ri ┆ mean     ┆ std      │
│ ---          ┆ ---   ┆ ---      ┆ ---      ┆   ┆ ght         ┆ ght         ┆ ---      ┆ ---      │
│ str          ┆ u32   ┆ u32      ┆ u32      ┆   ┆ ---         ┆ ---         ┆ f64      ┆ f64      │
│              ┆       ┆          ┆          ┆   ┆ f64         ┆ f64         ┆          ┆          │
╞══════════════╪═══════╪══════════╪══════════╪═══╪═════════════╪═════════════╪══════════╪══════════╡
│ 연구윤리_지  ┆ 7     ┆ 12       ┆ 61       ┆ … ┆ 0.45509     ┆ 0.065868    ┆ 3.431138 ┆ 0.881394 │
│ 침           ┆       ┆          ┆          ┆   ┆             ┆             ┆          ┆          │
│ 동물실험_윤  ┆ 21    ┆ 46       ┆ 56       ┆ … ┆ 0.221557    ┆ 0.041916    ┆ 2.778443 ┆ 1.060809 │
│ 리           ┆       ┆          ┆          ┆   ┆             ┆             ┆          

alt.HConcatChart(...)

In [227]:
# 
# 다. 국가연구개발혁신법 인지 여부
# 

columns = ["Q23"]

law_recognition = count_y("Q23")
law_recognition.write_csv("figure/4/다_국가혁신개발법_인지_여부.csv", include_bom=True)
chart = make_chart(columns)
chart.save("figure/4/다_국가혁신개발법_인지_여부.svg")

In [228]:
# 
# 라. 국가연구개발혁신법 파악 정도
# 

# Loop through each column and create a bar chart
columns = {
    "Q24_1": "연구진실성을_보호하기_위한_노력_및_관리체계", 
    "Q24_2": "학술지_투고_학회참석_등_학문교류에_관한_윤리", 
    "Q24_3": "이해충돌_예방_및_관리",
    "Q24_4": "인간대상_연구_및_동물실험에_관한_윤리",
    "Q24_5": "연구자의_권익보호_등_건전한_연구실_문화_조성",
}

count = count_list_y(columns.keys(), [1, 2, 3])

proportion = count.select(
    (pl.col("Q24_1") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_2") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_3") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_4") / (pl.col("Q24_1").sum())),
    (pl.col("Q24_5") / (pl.col("Q24_1").sum())),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

all = count.join(
    proportion, on="column", how="inner"
)

print(all)
all.write_csv("figure/4/라_국가혁신개발법_이해_여부.csv", include_bom=True)

charts = make_chart(columns)
charts.save("figure/4/라_국가혁신개발법_이해_여부.png")

shape: (5, 7)
┌─────────────────────┬───────┬──────────┬──────────┬────────────┬────────────────┬────────────────┐
│ column              ┆ count ┆ column_1 ┆ column_2 ┆ proportion ┆ column_1_right ┆ column_2_right │
│ ---                 ┆ ---   ┆ ---      ┆ ---      ┆ ---        ┆ ---            ┆ ---            │
│ str                 ┆ i64   ┆ i64      ┆ i64      ┆ f64        ┆ f64            ┆ f64            │
╞═════════════════════╪═══════╪══════════╪══════════╪════════════╪════════════════╪════════════════╡
│ 연구진실성을_보호하 ┆ 98    ┆ 2        ┆ 67       ┆ 0.586826   ┆ 0.011976       ┆ 0.401198       │
│ 기_위한_노력_및_관  ┆       ┆          ┆          ┆            ┆                ┆                │
│ 리체계              ┆       ┆          ┆          ┆            ┆                ┆                │
│ 학술지_투고_학회참  ┆ 106   ┆ 4        ┆ 57       ┆ 0.634731   ┆ 0.023952       ┆ 0.341317       │
│ 석_등_학문교류에_관 ┆       ┆          ┆          ┆            ┆                ┆                │
│ 한_윤리     

In [229]:
# 
# 마. 연구부정행위 위반 인지
# 

columns = {
    "Q25_1": "위조", 
    "Q25_2": "변조", 
    "Q25_3": "표절", 
    "Q25_4": "부당한 저자표시", 
    "Q25_5": "중복게재", 
    "Q25_6": "조사방해 행위, 제보자 위협 행위 등의 기타", 
    "Q25_7": "동물실험윤리", 
    "Q25_8": "연구노트 관리", 
    "Q25_9": "연구 성과 관리 및 연구 보안", 
    "Q25_10": "연구비 집행과 관리"
}

count = count_list_y(columns.keys(), [1, 2])

proportion = count.select(
    pl.col("Q25_1") / pl.col("Q25_1").sum(),
    pl.col("Q25_2") / pl.col("Q25_2").sum(),
    pl.col("Q25_3") / pl.col("Q25_3").sum(),
    pl.col("Q25_4") / pl.col("Q25_4").sum(),
    pl.col("Q25_5") / pl.col("Q25_5").sum(),
    pl.col("Q25_6") / pl.col("Q25_6").sum(),
    pl.col("Q25_7") / pl.col("Q25_7").sum(),
    pl.col("Q25_8") / pl.col("Q25_8").sum(),
    pl.col("Q25_9") / pl.col("Q25_9").sum(),
    pl.col("Q25_10") / pl.col("Q25_10").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})

all = count.join(
    proportion, on="column", how="inner"
)
print(all)
all.write_csv("figure/4/마_연구부정행위_위반_인지.csv", include_bom=True)

chart = make_chart(columns)
chart.save("figure/4/마_연구부정행위_위반_인지.svg")

shape: (10, 5)
┌─────────────────────────────────┬───────┬──────────┬────────────┬────────────────┐
│ column                          ┆ count ┆ column_1 ┆ proportion ┆ column_1_right │
│ ---                             ┆ ---   ┆ ---      ┆ ---        ┆ ---            │
│ str                             ┆ i64   ┆ i64      ┆ f64        ┆ f64            │
╞═════════════════════════════════╪═══════╪══════════╪════════════╪════════════════╡
│ 위조                            ┆ 10    ┆ 157      ┆ 0.05988    ┆ 0.94012        │
│ 변조                            ┆ 8     ┆ 159      ┆ 0.047904   ┆ 0.952096       │
│ 표절                            ┆ 9     ┆ 158      ┆ 0.053892   ┆ 0.946108       │
│ 부당한 저자표시                 ┆ 27    ┆ 140      ┆ 0.161677   ┆ 0.838323       │
│ 중복게재                        ┆ 14    ┆ 153      ┆ 0.083832   ┆ 0.916168       │
│ 조사방해 행위, 제보자 위협 행위 ┆ 5     ┆ 162      ┆ 0.02994    ┆ 0.97006        │
│ 등의 기타                       ┆       ┆          ┆            ┆                │

In [230]:
# 
# 바. 연구부정행위 위반 내용 보고 방법
# 

# 26. 귀하는 연구부정행위를 인지하게 되었을 때 KIOST 또는 외부의 관련 기관에 제보하는 방법을 알고 있습니까?
# 27. 귀하는 연구부정행위에 대한 제보가 접수된 후 어떻게 검증되는지 그 절차에 대해 알고 있습니까?
columns = {
    "Q26": "연구부정행위 제보 방법",
    "Q27": "연구부정행위 제보의 검증 절차"
}
count = count_list_y(columns.keys(), [1, 2])

proportion = count.select(
    pl.col("Q26") / pl.col("Q26").sum(),
    pl.col("Q27") / pl.col("Q27").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)
all.write_csv("figure/4/바_연구부정행위_위반_내용_보고_방법.csv", include_bom=True)
charts = make_chart(columns)
charts.save("figure/4/바_연구부정행위_위반_내용_보고_방법.png")

In [231]:
# 
# 사. KIOST의 연구부정행위에 대한 제보 접수 및 처리 규정 준수
# 
columns = {
    "Q28" : "연구부정행위 제보 접수 및 처리 규정 준수"
}
count = count_list_y(columns, [1, 2, 3, 4, 5])

proportion = count.select(
    pl.col("Q28") / pl.col("Q28").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)
all.write_csv("figure/4/사_KIOST의_연구부정행위에_대한_제보_접수_및_처리_규정_준수.csv", include_bom=True)

charts = make_chart(columns)
charts.save("figure/4/사_KIOST의_연구부정행위에_대한_제보_접수_및_처리_규정_준수.svg")

In [232]:
# 
# 아. KIOST의 연구부정행위 검증에 대한 객관성 및 공정성
# 
columns = {
    "Q29": "연구부정행위 검증에 대한 객관성 및 공정성"
}
count = count_list_y(columns, [1, 2, 3, 4, 5])

proportion = count.select(
    pl.col("Q29") / pl.col("Q29").sum(),
).rename(columns).transpose(include_header=True).rename({"column_0":"proportion"})

count = count.rename(columns).transpose(include_header=True).rename({"column_0":"count"})
all = count.join(
    proportion, on="column", how="inner"
)
all.write_csv("figure/4/아_KIOST의_연구부정행위_검증에_대한_객관성_및_공정성.csv", include_bom=True)
charts = make_chart(columns)
charts.save("figure/4/아_KIOST의_연구부정행위_검증에_대한_객관성_및_공정성.svg")

In [233]:
from attributes import *
from scipy import stats

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = research_ethics_system.join(personal_info, on="ID")

def run_chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
        chi_analysis[key] = {f"{key}_t":t, f"{key}_p":p}
    return chi_analysis

def run_chi(column, name):
    dataframe = pl.DataFrame()
    for c in column:
        result = pl.from_dict(run_chi2_test(c)) \
                    .unnest("sex") \
                    .unnest("age") \
                    .unnest("role") \
                    .unnest("degree") \
                    .unnest("experience") \
                    .unnest("field")
        dataframe = dataframe.vstack(result)
    dataframe.write_csv(f"figure/4/{name}_chi.csv", include_bom=True)


q21 = ["Q21_1", "Q21_2", "Q21_3", "Q21_4", "Q21_5"]
q22 = ["Q22_1", "Q22_2", "Q22_3", "Q22_4", "Q22_5"]
q23 = ["Q23"]
q24 = ["Q24_1", "Q24_2", "Q24_3", "Q24_4", "Q24_5"]
q25 = ["Q25_1", "Q25_2", "Q25_3", "Q25_4", "Q25_5", "Q25_6", "Q25_7", "Q25_8", "Q25_9", "Q25_10"]
q26 = ["Q26"]
q27 = ["Q27"]
q28 = ["Q28"]
q29 = ["Q29"]

run_chi(q21, "가_연구윤리_규정_인지_검정")
run_chi(q22, "나_연구윤리_규정_파악")
run_chi(q23, "다_국가혁신개발법_인지")
run_chi(q24, "라_국가혁신개발법_이해")
run_chi(q25, "마_연구부정행위_인지")
run_chi(q26, "바_연구부정행위_제보")
run_chi(q27, "바_연구부정행위_검증")
run_chi(q28, "사_연구부정행위_제보")
run_chi(q29, "아_연구부정행위_검증")

C:\Users\hyeon\AppData\Local\Temp\ipykernel_15396\4186764624.py:13: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
